In [1]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation
import numpy as np
import pandas as pd
from sklearn.svm import SVR



Using TensorFlow backend.


In [2]:
inputs = pd.read_csv('traincleaned.csv')
testinputs = pd.read_csv('testcleaned.csv')

inputs = inputs.drop(columns=['SalePrice'])
alldata = pd.concat([inputs, testinputs])
alldata.set_index('Id', inplace=True)
alldata = alldata.fillna(0)
alldata = pd.get_dummies(alldata)
alldata = alldata.drop(columns=['Unnamed: 0'])

inputs = alldata.loc[0:1460,]
inputs = inputs.values

testinputs = alldata.loc[1461:]
testinputs = testinputs.values



In [3]:
prices = pd.read_csv('train.csv')
prices = prices['SalePrice']
prices = np.array(prices)
prices

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [4]:
print(inputs.shape, prices.shape)

(1460, 311) (1460,)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


X_train, X_test, y_train, y_test = train_test_split(inputs, prices, random_state=1)

scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

X_train = X_train_scale
X_test = X_test_scale


In [6]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier


clf = SVR(kernel='rbf', C=90, gamma=0.2, epsilon=.5)
clf.fit(X_train, y_train) 

C:\Users\brand\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


SVR(C=90, cache_size=200, coef0=0.0, degree=3, epsilon=0.5, gamma=0.2,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [7]:
predictions = clf.predict(X_test)
predictions

array([166960.73884916, 164551.11037179, 164194.67800659, 164512.70959607,
       164470.49870039, 165343.00105092, 165456.07445543, 164604.366437  ,
       165501.17257174, 164772.67469617, 164587.86797763, 164579.54387781,
       165562.35984608, 164662.62295723, 165978.58281985, 164324.84792907,
       164311.06765553, 164249.35139657, 164677.32747909, 164493.18139766,
       164519.26742282, 164433.04063217, 165254.40860174, 164929.1444833 ,
       164526.80416368, 164582.49445239, 164568.02291937, 164791.58046646,
       164981.45606081, 164338.70489946, 164543.7069954 , 164188.75146309,
       164128.66638803, 164549.62206049, 164349.25658809, 165255.80880927,
       164134.90789309, 164535.50094574, 165013.63858658, 164484.64053355,
       164378.00976843, 164459.49857182, 164449.23873983, 164060.65800524,
       165304.67615761, 164504.13167898, 164138.64362459, 166018.58680195,
       165288.79203593, 164648.14068434, 164488.52346598, 164549.84309411,
       164488.99314452, 1

In [8]:
from sklearn.metrics import mean_squared_log_error
import math
math.sqrt(mean_squared_log_error(y_test, predictions))

0.40792784024230744

In [9]:
# for C in range(50,200,50):
    
#     for gamma in range(1, 10):
#         gamma = gamma/10
        
#         for epsilon in range(1,10):
#             epsilon = epsilon/10
            
#             clf = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
#             clf.fit(X_train, y_train) 
#             predictions = clf.predict(X_test)
#             error = 0
#             for x in range(100):
#                 error += abs(predictions[x] - y_test[x])/y_test[x]

#             print(C, gamma, epsilon, error/100)
#             break
            


In [ ]:
for C in range(50,200,20):
    
    for degree in range(4, 10):
        
        for epsilon in range(1,10, 2):
            epsilon = epsilon/10
            
            for coef0 in range(2,4):
            
                clf = SVR(kernel='poly', C=C, gamma='auto', degree=degree, epsilon=epsilon, coef0=coef0)
                clf.fit(X_train, y_train) 
                predictions = clf.predict(X_test)

                RMSLE = math.sqrt(mean_squared_log_error(y_test, predictions))
                
                if RMSLE < .14:
                    print(C, degree, epsilon, coef0, RMSLE)
                
                
#C=130,degree=7,epsilon=0.1,coef0=3 0.1359030945302552

50 8 0.1 3 0.13610449956675016
50 8 0.3 3 0.13610445913772304
50 8 0.5 3 0.13610442175844306
50 8 0.7 3 0.13610438762386537
50 8 0.9 3 0.13610435389039333
50 9 0.1 3 0.1373885508325944
50 9 0.3 3 0.13738843341020548
50 9 0.5 3 0.13738831597809265
50 9 0.7 3 0.13738819873439517
50 9 0.9 3 0.137388081483121


In [ ]:
clf = SVR(kernel='poly', C=130, gamma= 'auto', degree=7, epsilon=0.1, coef0=3)
clf.fit(X_train, y_train) 
testinputs = scaler.fit_transform(testinputs)
predictions = clf.predict(testinputs)



In [ ]:
submittest = pd.read_csv('testcleaned.csv')
submittest = submittest[['Id']]
submittest['SalePrice'] = predictions
submittest.to_csv('submission.csv', index=False)
submittest